# NIP-Polish-Tax-Number


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import string
from IPython.display import clear_output
import time

<b>Main_URL <b>: http://bnip.pl/

In [12]:
# Define list:
list = [str(i) for i in range(1, 10)] + list(string.ascii_uppercase)
list = [str(1)] 
list = [str(i) for i in range(1, 10)]
#list = ['A']

In [13]:
%%time

def scrape_detailed_info(session, url):
    """Detailed information from the given page. - scraper"""
    response = session.get(url)
    if response.encoding != 'ISO-8859-2':
        response.encoding = 'ISO-8859-2'
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        wszystkie_h1 = soup.find_all('h1')
        if len(wszystkie_h1) > 1:
            nazwa_firmy = wszystkie_h1[1].text.strip()
        else:
            nazwa_firmy = "Nazwa firmy nie znaleziona"
        
        # 
        nazwa_firmy = nazwa_firmy.replace('£', 'Ł').replace('¦', 'Ż').replace('¡', 'Ó')
        
        nip = soup.find('h2').text.strip()
        adres_element = soup.find('h2').find_next_siblings(text=True)
        adres = ''.join(adres_element).strip()
        return nazwa_firmy, nip, adres
    else:
        return "Błąd pobierania strony", "", ""

# HTTP session initialization
session = requests.Session()

# DataFrame initialization
full_df = pd.DataFrame()

for item in list:
    base_url = f'http://bnip.pl/nip,indexa,{item},{{}}.html'
    max_pages = None
    response = session.get(base_url.format(1))
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        p_elements = soup.find_all('p')
        for p in p_elements:
            matches = re.findall(r'(\d+)\<', p.text)
            if matches:
                max_pages = int(matches[-1])
    max_pages = max_pages or 1

    for page_number in range(1, max_pages + 1):
        # Information about the regular page
        print(f"Przetwarzanie: ({item}, strona {page_number} z {max_pages})")
        url = base_url.format(page_number)
        response = session.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table')
            if table:
                df = pd.read_html(str(table))[0]
                links = ['http://bnip.pl/' + a['href'] for a in soup.find_all('a', href=True) if 'a,nip,' in a['href']]
                df['Link'] = links[:len(df)]
                full_df = pd.concat([full_df, df], ignore_index=True)

for row in full_df.itertuples():
    url = getattr(row, 'Link')
    nazwa_firmy, nip, adres = scrape_detailed_info(session, url)

    full_df.at[row.Index, 'Nazwa Firmy'] = nazwa_firmy.encode('ISO-8859-2').decode('ISO-8859-2')
    full_df.at[row.Index, 'NIP'] = nip
    full_df.at[row.Index, 'Adres'] = adres

# Close session
session.close()


Przetwarzanie: (1, strona 1 z 64)
Przetwarzanie: (1, strona 2 z 64)
Przetwarzanie: (1, strona 3 z 64)
Przetwarzanie: (1, strona 4 z 64)
Przetwarzanie: (1, strona 5 z 64)
Przetwarzanie: (1, strona 6 z 64)
Przetwarzanie: (1, strona 7 z 64)
Przetwarzanie: (1, strona 8 z 64)
Przetwarzanie: (1, strona 9 z 64)
Przetwarzanie: (1, strona 10 z 64)
Przetwarzanie: (1, strona 11 z 64)
Przetwarzanie: (1, strona 12 z 64)
Przetwarzanie: (1, strona 13 z 64)
Przetwarzanie: (1, strona 14 z 64)
Przetwarzanie: (1, strona 15 z 64)
Przetwarzanie: (1, strona 16 z 64)
Przetwarzanie: (1, strona 17 z 64)
Przetwarzanie: (1, strona 18 z 64)
Przetwarzanie: (1, strona 19 z 64)
Przetwarzanie: (1, strona 20 z 64)
Przetwarzanie: (1, strona 21 z 64)
Przetwarzanie: (1, strona 22 z 64)
Przetwarzanie: (1, strona 23 z 64)
Przetwarzanie: (1, strona 24 z 64)
Przetwarzanie: (1, strona 25 z 64)
Przetwarzanie: (1, strona 26 z 64)
Przetwarzanie: (1, strona 27 z 64)
Przetwarzanie: (1, strona 28 z 64)
Przetwarzanie: (1, strona 29 

# Output

In [15]:
df_1 = full_df

In [16]:
df_1.to_excel("df_1.xlsx")